In [1]:
import os


In [2]:
pwd


'c:\\Users\\Devendra\\Projects\\LanguageTranslation\\research'

In [3]:
cd ..


c:\Users\Devendra\Projects\LanguageTranslation


In [4]:
pwd


'c:\\Users\\Devendra\\Projects\\LanguageTranslation'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from LanguageTranslation.constants import *
from LanguageTranslation.utils.utils import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
           
        )

        return model_evaluation_config

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm

c:\Users\Devendra\anaconda3\envs\LanguageTranslation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-01-25 19:30:16,479: INFO: config: PyTorch version 2.1.2 available.]
[2024-01-25 19:30:16,479: INFO: config: TensorFlow version 2.13.0 available.]


In [9]:
'''
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    
    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            parameter for length penalty ensures that the model does not generate sequences that are too long. 
            
            # Finally, we decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score


    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_opus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        #loading data 
        dataset_iitb_english_hindi_pt = load_from_disk(self.config.data_path)


        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
  
        rouge_metric = load_metric('rouge')

        score = self.calculate_metric_on_test_ds(
        dataset_iitb_english_hindi_pt['test'][0:10], rouge_metric, model_opus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
            )

        rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_name, index=False)

'''

'\nclass ModelEvaluation:\n    def __init__(self, config: ModelEvaluationConfig):\n        self.config = config\n\n\n    \n    def generate_batch_sized_chunks(self,list_of_elements, batch_size):\n        """split the dataset into smaller batches that we can process simultaneously\n        Yield successive batch-sized chunks from list_of_elements."""\n        for i in range(0, len(list_of_elements), batch_size):\n            yield list_of_elements[i : i + batch_size]\n\n    \n    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, \n                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", \n                               column_text="article", \n                               column_summary="highlights"):\n        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))\n        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))\n\n        for artic

In [20]:
import torch
from datasets import load_dataset, load_metric
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd

class ModelEvaluation:
    def __init__(self, config):
        self.config = config
    
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]
    
    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer, 
                                    batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                                    column_input="input_text", 
                                    column_target="target_text"):
        input_batches = list(self.generate_batch_sized_chunks(dataset[column_input], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_target], batch_size))

        for input_batch, target_batch in tqdm(zip(input_batches, target_batches), total=len(input_batches)):
            inputs = tokenizer(input_batch, max_length=128,  truncation=True, 
                               padding="max_length", return_tensors="pt")
            
            outputs = model.generate(input_ids=inputs["input_ids"].to(device),
                                     attention_mask=inputs["attention_mask"].to(device), 
                                     length_penalty=0.8, num_beams=8, max_length=128)
            
            decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) for output in outputs]
            
            #decoded_outputs = [d.replace("", " ") for d in decoded_outputs]
            
            metric.add_batch(predictions=decoded_outputs, references=target_batch)
            
        score = metric.compute()
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        # Load data 
        dataset_iitb_english_hindi_pt = load_dataset(self.config.data_path)

        bleu_metric = load_metric('bleu')

        score = self.calculate_metric_on_test_ds(
            dataset_iitb_english_hindi_pt['test'][0:10], bleu_metric, model, tokenizer, batch_size=2,
            column_input='input_text', column_target='target_text'
        )

        bleu_score = score['score']

        df = pd.DataFrame({"BLEU": [bleu_score]})
        df.to_csv(self.config.metric_file_name, index=False)


In [21]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2024-01-25 19:39:47,311: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2024-01-25 19:39:47,311: INFO: utils: yaml file: params.yaml loaded successfully]
[2024-01-25 19:39:47,319: INFO: utils: created directory at: artifacts]
[2024-01-25 19:39:47,319: INFO: utils: created directory at: artifacts/model_evaluation]


ValueError: Couldn't infer the same data file format for all splits. Got {NamedSplit('train'): ('arrow', {}), NamedSplit('validation'): ('json', {}), NamedSplit('test'): ('json', {})}